# Replicación Artículo: Forecasting Short Term Photovoltaic Power Model Based on Improved Snake Optimization, Convolutional Neural Network and Bidirectional Long Short-Term Memory Network

In [99]:
import warnings
warnings.filterwarnings("ignore")

Leemos los datos

In [100]:
import pandas as pd
datos = pd.read_excel("02.Generación Pearson.xlsx")
datos.head(4)

,Fecha,Generación,Índice UV,Temperatura,Hora,DPT,Cobertura Nubes,Condición Cielo,Humedad Relativa
0,2022-09-01 00:00:00,0.0,0.0,19,24,15,98,4,77
1,2022-09-01 01:00:00,0.0,0.0,19,1,16,82,4,82
2,2022-09-01 02:00:00,0.0,0.0,18,2,16,73,3,85
3,2022-09-01 03:00:00,0.0,0.0,18,3,16,63,3,87


In [101]:
datos_2 = datos.copy()
datos_2["Generacion_prev_hour"] = datos_2["Generación"].shift(1)
datos_dropna = datos_2.dropna(how="any", axis= 0)

In [102]:
datos_dropna.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18289 entries, 1 to 18289
Data columns (total 10 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   Fecha                 18289 non-null  datetime64[ns]
 1   Generación            18289 non-null  float64       
 2   Índice UV             18289 non-null  float64       
 3   Temperatura           18289 non-null  int64         
 4   Hora                  18289 non-null  int64         
 5   DPT                   18289 non-null  int64         
 6   Cobertura Nubes       18289 non-null  int64         
 7   Condición Cielo       18289 non-null  int64         
 8   Humedad Relativa      18289 non-null  int64         
 9   Generacion_prev_hour  18289 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(6)
memory usage: 1.5 MB


In [103]:
datos_sinfecha = datos_dropna.drop(["Fecha"], axis=1)

In [104]:
datos_sinfecha.info()

<class 'pandas.core.frame.DataFrame'>
Index: 18289 entries, 1 to 18289
Data columns (total 9 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   Generación            18289 non-null  float64
 1   Índice UV             18289 non-null  float64
 2   Temperatura           18289 non-null  int64  
 3   Hora                  18289 non-null  int64  
 4   DPT                   18289 non-null  int64  
 5   Cobertura Nubes       18289 non-null  int64  
 6   Condición Cielo       18289 non-null  int64  
 7   Humedad Relativa      18289 non-null  int64  
 8   Generacion_prev_hour  18289 non-null  float64
dtypes: float64(3), int64(6)
memory usage: 1.4 MB


In [105]:
datos_sinfecha.describe().T

,count,mean,std,min,25%,50%,75%,max
Generación,18289.0,9392.983969,11398.764291,0.0,0.0,533.0,21830.0,30000.0
Índice UV,18289.0,1.976418,3.181404,0.0,0.0,0.0,3.0,14.0
Temperatura,18289.0,19.456723,6.636719,0.0,15.0,19.0,24.0,39.0
Hora,18289.0,12.500683,6.922804,1.0,7.0,13.0,19.0,25.0
DPT,18289.0,9.837881,4.803705,0.0,6.0,10.0,14.0,20.0
Cobertura Nubes,18289.0,50.305594,37.864498,0.0,12.0,48.0,92.0,100.0
Condición Cielo,18289.0,2.675816,1.493514,1.0,1.0,3.0,4.0,5.0
Humedad Relativa,18289.0,58.878233,26.112575,3.0,37.0,62.0,83.0,100.0
Generacion_prev_hour,18289.0,9392.983969,11398.764291,0.0,0.0,533.0,21830.0,30000.0


Normalización de datos mediante MinMaxScaler

In [106]:
from timeit import default_timer as timer
from sklearn.preprocessing import MinMaxScaler
scaler = MinMaxScaler()

In [107]:
X = datos_sinfecha.drop('Generación', axis=1)
X_scaled = scaler.fit_transform(X)
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)
X = X_scaled_df.copy()
y = datos_sinfecha['Generación']

In [108]:
X.sample(4)

,Índice UV,Temperatura,Hora,DPT,Cobertura Nubes,Condición Cielo,Humedad Relativa,Generacion_prev_hour
9884,0.000000,0.538462,0.791667,0.25,1.00,1.00,0.340206,0.004633
1883,0.142857,0.358974,0.416667,0.45,0.95,0.75,0.690722,0.034600
9475,0.000000,0.692308,0.750000,0.50,0.42,0.50,0.329897,0.500033
1105,0.000000,0.435897,0.041667,0.50,0.49,0.50,0.628866,0.000000


Agrupación de datos en soleado, nublado y lluvioso

In [109]:
from sklearn.cluster import KMeans
k = 3
kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
clusters = kmeans.fit_predict(X)

X_orig = pd.DataFrame(scaler.inverse_transform(X), index=X.index, columns=X.columns)

datos_sinfecha['Cluster'] = clusters

print(datos_sinfecha['Cluster'].value_counts())

Cluster
2    7844
1    5321
0    5124
Name: count, dtype: int64


In [110]:
datos_sinfecha

,Generación,Índice UV,Temperatura,Hora,DPT,Cobertura Nubes,Condición Cielo,Humedad Relativa,Generacion_prev_hour,Cluster
1,0.0,0.0,19,1,16,82,4,82,0.0,2
2,0.0,0.0,18,2,16,73,3,85,0.0,2
3,0.0,0.0,18,3,16,63,3,87,0.0,2
4,0.0,0.0,18,4,16,70,3,88,0.0,2
5,0.0,0.0,17,5,14,45,3,86,0.0,2
...,...,...,...,...,...,...,...,...,...,...
18285,0.0,0.0,22,20,9,2,1,45,1450.0,0
18286,0.0,0.0,20,21,10,1,1,54,0.0,0
18287,0.0,0.0,18,22,11,0,1,62,0.0,0
18288,0.0,0.0,17,23,11,0,1,69,0.0,0


In [111]:
# Calcular estadísticas de cada cluster
cluster_means = datos_sinfecha.groupby('Cluster').mean()
cluster_means


,Generación,Índice UV,Temperatura,Hora,DPT,Cobertura Nubes,Condición Cielo,Humedad Relativa,Generacion_prev_hour
Cluster,,,,,,,,,
0,2857.485022,0.536261,17.377830,12.595628,9.636222,13.077674,1.299571,62.496682,1887.758629
1,22823.803330,5.391806,26.029694,14.700808,8.235294,35.382635,1.930652,33.651193,24651.839869
2,4551.386159,0.600344,16.355941,10.946201,11.056731,84.747323,4.080316,73.627358,3944.794575


Definimos el modelo

In [52]:
pip install tensorflow

  Using cached absl_py-2.1.0-py3-none-any.whl.metadata (2.3 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl.metadata (4.4 kB)
  Using cached google_pasta-0.2.0-py3-none-any.whl.metadata (814 bytes)
  Using cached libclang-18.1.1-py2.py3-none-win_amd64.whl.metadata (5.3 kB)
  Using cached namex-0.0.8-py3-none-any.whl.metadata (246 bytes)
  Using cached tensorboard_data_server-0.7.2-py3-none-any.whl.metadata (1.1 kB)
  Using cached markdown_it_py-3.0.0-py3-none-any.whl.metadata (6.9 kB)
  Using cached mdurl-0.1.2-py3-none-any.whl.metadata (1.6 kB)
   ---------------------------------------- 0.0/390.0 MB ? eta -:--:--
    --------------------------------------- 5.8/390.0 MB 35.3 MB/s eta 0:00:11
   - -------------------------------------- 12.1/390.0 MB 30.2 MB/s eta 0:00:13
   -- ------------------------------------- 26.5/390.0 MB 40.0 MB/s eta 0:00:10
   ---- ----------------------------------- 39.1/390.0 MB 45.1 MB/s eta 0:00:08
   ----- ---------------------------------- 54.3/

In [54]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("GPU available:", tf.config.list_physical_devices('GPU'))

TensorFlow version: 2.18.0
GPU available: []


In [123]:
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense, LSTM, Bidirectional, Dropout, BatchNormalization, Activation, Reshape

In [118]:
clusters_col = datos_sinfecha['Cluster']
X = datos_sinfecha.drop(['Generación', 'Cluster'], axis = 1)
scaler = MinMaxScaler()
X_scaled = scaler.fit_transform(X)

# Restaurar en un DataFrame con los nombres originales
X_scaled_df = pd.DataFrame(X_scaled, columns=X.columns)

# Volver a agregar la columna 'Clusters' al DataFrame escalado
X_scaled_df['Cluster'] = clusters_col.values

# Copia final de X y y
X = X_scaled_df.copy()
y = datos_sinfecha['Generación']

In [119]:
X

,Índice UV,Temperatura,Hora,DPT,Cobertura Nubes,Condición Cielo,Humedad Relativa,Generacion_prev_hour,Cluster
0,0.0,0.487179,0.000000,0.80,0.82,0.75,0.814433,0.000000,2
1,0.0,0.461538,0.041667,0.80,0.73,0.50,0.845361,0.000000,2
2,0.0,0.461538,0.083333,0.80,0.63,0.50,0.865979,0.000000,2
3,0.0,0.461538,0.125000,0.80,0.70,0.50,0.876289,0.000000,2
4,0.0,0.435897,0.166667,0.70,0.45,0.50,0.855670,0.000000,2
...,...,...,...,...,...,...,...,...,...
18284,0.0,0.564103,0.791667,0.45,0.02,0.00,0.432990,0.048333,0
18285,0.0,0.512821,0.833333,0.50,0.01,0.00,0.525773,0.000000,0
18286,0.0,0.461538,0.875000,0.55,0.00,0.00,0.608247,0.000000,0
18287,0.0,0.435897,0.916667,0.55,0.00,0.00,0.680412,0.000000,0


In [143]:
timesteps = 3

features = X.columns.drop('Cluster')
target = 'Generación'

# División temporal en 80-20
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

# Restablecer los índices para evitar problemas de alineación
X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

# Crear diccionarios para almacenar datos por cluster
X_clusters_train, y_clusters_train = {}, {}
X_clusters_test, y_clusters_test = {}, {}

# Iterar sobre cada cluster único en los datos de entrenamiento
for cluster in X['Cluster'].unique():
    df_train_cluster = X_train[X_train['Cluster'] == cluster].copy()
    df_test_cluster = X_test[X_test['Cluster'] == cluster].copy()
    
    X_train_seq, y_train_seq = [], []
    X_test_seq, y_test_seq = [], []

    # Construir secuencias para entrenamiento
    for i in range(len(df_train_cluster) - timesteps):
        X_train_seq.append(df_train_cluster[features].iloc[i:i+timesteps].values)
        y_train_seq.append(y_train.iloc[i+timesteps])  # Valor objetivo desplazado

    # Construir secuencias para prueba
    for i in range(len(df_test_cluster) - timesteps):
        X_test_seq.append(df_test_cluster[features].iloc[i:i+timesteps].values)
        y_test_seq.append(y_test.iloc[i+timesteps])

    # Convertir a NumPy
    X_train_seq, y_train_seq = np.array(X_train_seq), np.array(y_train_seq)
    X_test_seq, y_test_seq = np.array(X_test_seq), np.array(y_test_seq)

    # Almacenar en diccionarios
    X_clusters_train[cluster] = X_train_seq
    y_clusters_train[cluster] = y_train_seq
    X_clusters_test[cluster] = X_test_seq
    y_clusters_test[cluster] = y_test_seq

    print(f"Cluster {cluster} - X_train: {X_train_seq.shape}, X_test: {X_test_seq.shape}")

Cluster 2 - X_train: (6281, 3, 8), X_test: (1557, 3, 8)
Cluster 1 - X_train: (4149, 3, 8), X_test: (1166, 3, 8)
Cluster 0 - X_train: (4192, 3, 8), X_test: (926, 3, 8)


In [145]:
models = {}

for cluster in X_clusters_train.keys():
    print(f"Entrenando modelo para Cluster {cluster}...")

    X_train, X_test = X_clusters_train[cluster], X_clusters_test[cluster]
    y_train, y_test = y_clusters_train[cluster], y_clusters_test[cluster]

    model = Sequential([
        Conv1D(filters=64, kernel_size=3, padding='same', input_shape=(X_train.shape[1], X_train.shape[2])),
        BatchNormalization(),
        Activation('relu'),
        MaxPooling1D(pool_size=2),

        Bidirectional(LSTM(50, return_sequences=False)),

        Dense(50, activation='relu'),
        Dropout(0.2),
        
        # Output layer
        Dense(1, activation='linear')
    ])

    model.compile(optimizer='adam', loss='mse', metrics=['mae'])
    # Entrenar el modelo
    model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_test, y_test))
    
    # Evaluar el modelo
    test_loss, test_mae = model.evaluate(X_test, y_test)
    print(f'Cluster {cluster} - Test Loss: {test_loss}, Test MAE: {test_mae}')
    
    # Guardar el modelo entrenado para cada cluster
    models[cluster] = model

Entrenando modelo para Cluster 2...
Epoch 1/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 8s 11ms/step - loss: 184599440.0000 - mae: 8149.5083 - val_loss: 275849696.0000 - val_mae: 11416.3760
Epoch 2/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 190294976.0000 - mae: 8500.6621 - val_loss: 265397584.0000 - val_mae: 11350.1826
Epoch 3/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 6ms/step - loss: 173041904.0000 - mae: 8223.1172 - val_loss: 250569712.0000 - val_mae: 11277.2764
Epoch 4/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 169297312.0000 - mae: 8512.2393 - val_loss: 233338720.0000 - val_mae: 11232.7568
Epoch 5/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 1s 7ms/step - loss: 159811072.0000 - mae: 8671.6152 - val_loss: 216001504.0000 - val_mae: 11235.5527
Epoch 6/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 10ms/step - loss: 144081296.0000 - mae: 8610.8223 - val_loss: 200050512.0000 - val_mae: 11254.3252
Epoch 7/50
197/197 ━━━━━━━━━━━━━━━━━━━━ 2s 8ms/step - loss: 140961760.0000 - mae: 9050.5469 - val_loss: 187062496.00

In [ ]:
import random

# Parámetros de Snake Optimization
num_individuals = 10  # Tamaño de la población
num_iterations = 20  # Número de iteraciones
search_space = {
    "filters": [32, 64, 128],
    "lstm_units": [50, 100, 150],
    "dropout_rate": [0.2, 0.3, 0.4],
    "kernel_size": [3, 5],
    "learning_rate": [0.001, 0.0005, 0.0001],
    "epochs": [30, 50, 100],
    "batch_size": [16, 32, 64]
}

# Inicializar población usando Tent Chaotic Mapping
population = [
    {
        key: random.choice(values) for key, values in search_space.items()
    }
    for _ in range(num_individuals)
]

best_snake = None
best_loss = float("inf")

# Ejecución del algoritmo Snake Optimization
for cluster in range:
    print(f"Optimización para Cluster {cluster}...")
    X_train, X_test = X_clusters_train[cluster], X_clusters_test[cluster]
    y_train, y_test = y_clusters_train[cluster], y_clusters_test[cluster]
    
    for iteration in range(num_iterations):
        print(f"Iteración {iteration + 1}/{num_iterations}")

        #Dividir entre Machos y Hembras
        male_population = population[:len(population)//2]
        female_population = population[:len(population)//2:]

        for individual in population:
            model = Sequential([
                Conv1D(filters=individual["filters"], kernel_size=individual["kernel_size"], padding='same', input_shape=(X_train.shape[1], X_train.shape[2])),
                BatchNormalization(),
                Activation('relu'),
                MaxPooling1D(pool_size=2),
                Bidirectional(LSTM(individual["lstm_units"], return_sequences=False)),
                Dense(50, activation='relu'),
                Dropout(individual["dropout_rate"]),
                Dense(1, activation='linear')
            ])

            optimizer = tf.keras.optimizers.Adam(learning_rate=individual["learning_rate"])
            model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

            model.fit(X_train, y_train, epochs=individual["epochs"], batch_size=individual["batch_size"], verbose=0)
            loss, mae = model.evaluate(X_test, y_test, verbose=0)

            if loss < best_loss:
                best_loss = loss
                best_solution = individual

        # Aplicar fases del MISO
        Q = random.random()
        Temp = random.random()
        Rand = random.random()

        if Q < 0.2:
            # Exploración (movimiento aleatorio)
            for ind in population:
                ind["filters"] = random.choice(search_space["filters"])
        elif Temp > 0.6:
            # Explotación (Mejorar individuos usando cruzamiento)
            for i in range(len(female_population)):
                female_population[i]["dropout_rate"] = male_population[i]["dropout_rate"]
        elif Rand < 0.6:
            # Modo de pelea (competencia entre individuos)
            for ind in population:
                ind["epochs"] = random.choice(search_space["epochs"])
                ind["batch_size"] = random.choice(search_space["batch_size"])

    # Mejor combinación de hiperparámetros encontrada para el cluster
    print(f"Mejores hiperparámetros para Cluster {cluster}: {best_solution}")
    



Optimización para Cluster 2...
Iteración 1/20
Iteración 2/20
Iteración 3/20
Iteración 4/20
Iteración 5/20
Iteración 6/20
Iteración 7/20
Iteración 8/20
Iteración 9/20
Iteración 10/20
Iteración 11/20
Iteración 12/20
Iteración 13/20
Iteración 14/20
Iteración 15/20
Iteración 16/20
Iteración 17/20
Iteración 18/20
Iteración 19/20
Iteración 20/20
Mejores hiperparámetros para Cluster 2: {'filters': 64, 'lstm_units': 100, 'dropout_rate': 0.3, 'kernel_size': 3, 'learning_rate': 0.0005, 'epochs': 30, 'batch_size': 16}


ValueError: Input 0 of layer "bidirectional_206" is incompatible with the layer: expected ndim=3, found ndim=2. Full shape received: (None, 64)

In [ ]:
import random

# Parámetros de Snake Optimization
num_individuals = 10  # Tamaño de la población
num_iterations = 20  # Número de iteraciones
search_space = {
    "filters": [32, 64, 128],
    "lstm_units": [50, 100, 150],
    "dropout_rate": [0.2, 0.3, 0.4],
    "kernel_size": [3, 5],
    "learning_rate": [0.001, 0.0005, 0.0001],
    "epochs": [30, 50, 100],
    "batch_size": [16, 32, 64]
}

# Inicializar población usando Tent Chaotic Mapping
population = [
    {
        key: random.choice(values) for key, values in search_space.items()
    }
    for _ in range(num_individuals)
]

best_snake = None
best_loss = float("inf")

# Ejecución del algoritmo Snake Optimization
cluster = 1
print(f"Optimización para Cluster {cluster}...")
X_train, X_test = X_clusters_train[cluster], X_clusters_test[cluster]
y_train, y_test = y_clusters_train[cluster], y_clusters_test[cluster]

for iteration in range(num_iterations):
    print(f"Iteración {iteration + 1}/{num_iterations}")

    #Dividir entre Machos y Hembras
    male_population = population[:len(population)//2]
    female_population = population[:len(population)//2:]

    for individual in population:
        model = Sequential([
            Conv1D(filters=individual["filters"], kernel_size=individual["kernel_size"], padding='same', input_shape=(X_train.shape[1], X_train.shape[2])),
            BatchNormalization(),
            Activation('relu'),
            MaxPooling1D(pool_size=2),
            Bidirectional(LSTM(individual["lstm_units"], return_sequences=False)),
            Dense(50, activation='relu'),
            Dropout(individual["dropout_rate"]),
            Dense(1, activation='linear')
        ])

        optimizer = tf.keras.optimizers.Adam(learning_rate=individual["learning_rate"])
        model.compile(optimizer=optimizer, loss='mse', metrics=['mae'])

        model.fit(X_train, y_train, epochs=individual["epochs"], batch_size=individual["batch_size"], verbose=0)
        loss, mae = model.evaluate(X_test, y_test, verbose=0)

        if loss < best_loss:
            best_loss = loss
            best_solution = individual

    # Aplicar fases del MISO
    Q = random.random()
    Temp = random.random()
    Rand = random.random()

    if Q < 0.2:
        # Exploración (movimiento aleatorio)
        for ind in population:
            ind["filters"] = random.choice(search_space["filters"])
    elif Temp > 0.6:
        # Explotación (Mejorar individuos usando cruzamiento)
        for i in range(len(female_population)):
            female_population[i]["dropout_rate"] = male_population[i]["dropout_rate"]
    elif Rand < 0.6:
        # Modo de pelea (competencia entre individuos)
        for ind in population:
            ind["epochs"] = random.choice(search_space["epochs"])
            ind["batch_size"] = random.choice(search_space["batch_size"])

# Mejor combinación de hiperparámetros encontrada para el cluster
print(f"Mejores hiperparámetros para Cluster {cluster}: {best_solution}")
    



Optimización para Cluster 1...
Iteración 1/20
